In [1]:
# 문서 출력 도우미 함수
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

# 기본 Retriever

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter




# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
# 문서 로드
loader = TextLoader("./data/text.txt", encoding="utf-8")
# 문서 분할
split_docs = loader.load_and_split(text_splitter)

# 임베딩 모델 설정
model_name = "intfloat/multilingual-e5-large-instruct"
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cuda"},  # cuda, cpu
    encode_kwargs={"normalize_embeddings": True},
)

# 문서로부터 FAISS
db = FAISS.from_documents(documents=split_docs, embedding=hf_embeddings)

    # 5. 리트리버 생성
retriever = db.as_retriever(
    search_type="mmr", search_kwargs={"k": 10, "lambda_mult": 0.25, "fetch_k": 30}
)

# 질의 설정
query = "삼성전자 하반기 실적 전망은?"

# 질의 수행 및 결과 문서 반환
docs = retriever.invoke(query)

# 결과 문서 출력
pretty_print_docs(docs)

Document 1:

특히 파운드리 고객사를 확보한 것이 단기 실적에는 영향을 미치지 않지만, 중장기 경쟁력 회복을 위한 기반이 될 것이라고 평가했다. 류 연구원은 "여러 기대감이 모여 밸류에이션이 정상화되는 국면"이라고 분석했다. 
이어 3분기 영업이익은 9조원으로 1년 전보다는 1.9% 감소하지만 직전 분기 대비 91.9% 늘어날 것으로 낙관했다.
미래에셋증권은 15일 보고서에서 삼성전자의 올해와 내년 영업이익 전망치를 기존대비 각각 2.9%, 5.3%씩 올렸다. 동시에 목표 주가를 기존 8만8000원에서 9만6000원으로 9% 높였다.
미래에셋증권은 주가 상향 이유로 내년 메모리 반도체 공급 부족을 꼽았다. 반도체 업체들은 제한된 D램 생산라인을 갖고 있는데, 업황 불확실성 탓에 설비를 크게 늘리지 못했다. 새로 증설한 시설도 대부분 차세대 반도체인 고대역폭 메모리(HBM) 생산에 집중돼 있어 범용 D램의 생산량이 줄면서 가격이 오르고 있다.
----------------------------------------------------------------------------------------------------
Document 2:

일탈 조항
국제회계기준 IFRS17의 원칙적인 적용이 어려운 특수한 상황에서 예외적인 회계처리를 허용하는 규정. 일탈 조항의 적용을 받지 못하게 되면 삼성생명 측은 향후 삼성전자 주식을 어떻게 매각해서 계약자에게 돌려줘야 할지 계획을 세워야 하는 상황에 몰리게 된다.
[박창영 기자]"절차 정당성·구속력에 문제"
국제회계기준 부합도 따져야
삼성생명의 회계처리 방식과 관련해 한국회계기준원이 적용의견서 발표에 속도를 내는 가운데 회계학 전문가들 사이에서는 절차적 정당성이나 국제호계기준과 합치 여부, 기업과 시장의 혼란 가능성 등에 대한 우려도 나온다.
-----------------------------------------------------------------------------------------------

In [ ]:
from langchain_community.document_transformers import LongContextReorder

# 문서를 재정렬합니다
# 덜 관련된 문서는 목록의 중간에 위치하고 더 관련된 요소는 시작/끝에 위치합니다.
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# 4개의 관련 문서가 시작과 끝에 위치하는지 확인합니다.
pretty_print_docs(reordered_docs)

Document 1:

일탈 조항
국제회계기준 IFRS17의 원칙적인 적용이 어려운 특수한 상황에서 예외적인 회계처리를 허용하는 규정. 일탈 조항의 적용을 받지 못하게 되면 삼성생명 측은 향후 삼성전자 주식을 어떻게 매각해서 계약자에게 돌려줘야 할지 계획을 세워야 하는 상황에 몰리게 된다.
[박창영 기자]"절차 정당성·구속력에 문제"
국제회계기준 부합도 따져야
삼성생명의 회계처리 방식과 관련해 한국회계기준원이 적용의견서 발표에 속도를 내는 가운데 회계학 전문가들 사이에서는 절차적 정당성이나 국제호계기준과 합치 여부, 기업과 시장의 혼란 가능성 등에 대한 우려도 나온다.
----------------------------------------------------------------------------------------------------
Document 2:

'스캐터드 랩서스$'는 국내 기업뿐 아니라 대만 통신사 중화 텔레콤 관련 정보 1.7TB(테라바이트), 인도 신분증·여권 정보, 구찌 등의 고객 정보를 갖고 있다고 주장했다. 이들은 삼성전자와 LG전자, MS, 엔비디아 등을 해킹한 해커그룹 '랩서스'와 유사한 이름을 썼지만 같은 집단인지는 확인되지 않았다.과기정통부와 한국인터넷진흥원(KISA)도 SKT에 관련 자료 제출을 요구했으며 현장점검 등을 통한 사실관계를 파악할 계획이라고 밝혔다. 과기정통부 관계자는 "최근 늘어나는 침해 사고로 인해 국민 불안이 가중되고 있다"며 "관련 주장에 대해 신속하게 사실관계를 확인하고 결과를 국민에게 투명하게 공개하겠다"고 설명했다.이재용 주식자산 16일 19조152억원
삼성전자 7만9000원 선 뚫은 덕분 
외국인 매수세 삼성 계열사 주가 상승 긍정적 영향
 미소 보이는 이재용 삼성전자 회장. 연합뉴스
----------------------------------------------------------------------------------------------------
Document 3:

16일

: 